# Data setup

In [2]:
# initialize data 
###
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# plot table
import texttable as tt

def plotTable(headers,
              rows,
              valign=True,
              column_width=False,
              no_deco=False,
              precision=1):
    tab = tt.Texttable()
    if len(headers) > 0:
        tab.header(headers)
    tab.set_precision(precision)
    for row in rows:
        tab.add_row(row)

    if valign and len(headers) > 0:
        tab.set_cols_valign(['m'] * len(headers))

    if column_width:
        columns = [0] * len(rows[0])
        for row in rows:
            for i, cell in enumerate(row):
                if isinstance(row[i], str):
                    columns[i] = max(
                        columns[i],
                        max([len(line) for line in row[i].split('\n')]))
        for i, head in enumerate(headers):
            columns[i] = max(columns[i], len(head))
        tab.set_cols_width(columns)

    if no_deco:
        tab.set_deco(tt.Texttable.HEADER | tt.Texttable.VLINES)

    s = tab.draw()
    return s

def data_from_file(file_name):
    #case = 1-> Id from file. Iq in 0->220 > 'i_direct_flux'
    #case = 2 -> Iq from file. Id in 0->220 > 'j_q_flux'
    data = np.loadtxt(file_name)
    alfa = list(range(0,31))
    flux = data[0:, :]
    I_currents = [0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6]
    xLabel = " I [A]"
    yLabel = "Flux"
    return alfa, flux, I_currents, xLabel, yLabel
    
alfa, Flux, I_currents, xLabel, yLabel = data_from_file('flux_s.txt')


# Represent original curves

In [ ]:
#

from subprocess import call
plt.clf()
plt.rcParams["figure.figsize"] = [30,30]
plt.rcParams['xtick.labelsize']=14
plt.rcParams['ytick.labelsize']=14

flux = Flux

for idx in alfa:
    df =flux[:, idx]    
    plt.plot(I_currents, df, label='alfa = %d grades' %(idx))
    plt.ylim([0,0.35])
    plt.xlim([0, 3])
    plt.xticks(I_currents)
    plt.xlabel(xLabel, fontsize = 16)
    plt.ylabel(yLabel, fontsize = 16)
    plt.legend(loc='best', fontsize = 16)
    plt.scatter(I_currents,df)
plt.grid()
plt.savefig('SRM_Original_curves.png', dpi = 200)
plt.show()

#  PCA to reduce the number of defining point  (ids from Ox axis)

    
Ideas:
 - Reduce the number of points, using PCA
 - Pt. reconstructie:
     - Spline pe nr redus de puncte
     - Loss intre Spline si curbele initiale
     
 


In [ ]:
#
import numpy as np
from scipy.interpolate import interp1d
from scipy import interpolate
from sklearn.decomposition import PCA
from sklearn import preprocessing

#INITIALIZE THE TABLE 
headers =[ 'PCs' ] + ['Iq %s'%id for id in I_currents] 
rows    = []

#preprocess the data
flux = Flux
scaler = preprocessing.StandardScaler()
df_flux_scaled = scaler.fit_transform(flux)

curve_indices = list(range(31))    
no_of_comps = [2,4,6,8,10,12,13]

error_per_PCs = np.zeros(shape = len(no_of_comps))
error_idx = 0

for comp in no_of_comps:
    
    row = [comp]
    pca_reduce_points = PCA(n_components=comp)
    flux_reduced = pca_reduce_points.fit_transform(df_flux_scaled)
    projected  = pca_reduce_points.inverse_transform(flux_reduced)
    projected_unscaled = scaler.inverse_transform(projected)
    
    for curve in curve_indices:
    #spline for the reduced no of points
        x = I_currents
        y = flux[:,curve]

        tck = interpolate.splrep(x, y, s=0)
        x_spline = np.linspace(min(I_currents), max(I_currents), 200)
        y_spline = interpolate.splev(x_spline, tck, der=0)
                                     
        x_new = np.linspace(projected_unscaled.min(), projected_unscaled.max(), 200)
        projected_baseline = interpolate.splev(x_new, tck, der=0)
        
        error = ((y_spline - projected_baseline) ** 2).sum()
        error = error.astype(np.float64) * (10 ** 6)
        row.append(error)
        error_per_PCs[error_idx]+=error
    error_idx+=1
    rows.append(row)
      
    plt.clf()
    plt.rcParams["figure.figsize"] = [12,8]
    plt.rcParams['xtick.labelsize']=12
    plt.rcParams['ytick.labelsize']=12
    
    components = pca_reduce_points.components_
    components = scaler.inverse_transform(components)
    

    headers_PC =[ 'Idx of PC' ] + ['I %s'%id for id in I_currents] 
    rows_PC    = []
    i = 1
    for component in pca_reduce_points.components_:
        row_PC = [i]
        for value in component:
            row_PC.append(value)
        i+=1
        rows_PC.append(row_PC)
        
#visualize the errors per PCs
# plt.clf()
plt.rcParams["figure.figsize"] = [12, 12]
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12
plt.xlabel('No of Principal Components', fontsize = 14)
plt.ylabel('Error of # Principal Components ', fontsize = 14)
plt.xlim([0,14])
plt.scatter(no_of_comps, error_per_PCs)
plt.plot(no_of_comps, error_per_PCs)
plt.grid()
plt.savefig('SRM_errorsPCSPoints.png', dpi = 200)
plt.show()
    
    
# # print PC meaning   
# print ("Meaning of the %d components(Weights of each Id from OX axis)"  %comp)
# table_PC = plotTable(headers=headers_PC, rows=rows_PC, column_width=True, precision=5)
# print(table_PC)

# # print errors in table    
# print('Loss ( sum squared error * 10 ^ 6)')
# table = plotTable(headers=headers, rows=rows, column_width=True, precision=4)
# print(table)
# print(error_per_PCs)

# Ridge for PCA reducing points no - reducing Ids from OX


In [ ]:
#
from sklearn.linear_model import Ridge
from pandas import DataFrame as df

X = Flux
y = I_currents

model = Ridge(fit_intercept=True, normalize=True)
model.fit(X, y)

# best case 1, worst case -1, random prediction: 0
print('Regression train score: %.2f\n' % model.score(X, y))

components = pca_reduce_points.components_
components = scaler.inverse_transform(components)

#find upper and lower Iq curve
def border_curves(curve, interval):
    left = max([i for i in interval if i < curve])
    right = min([i for i in interval if i > curve])

    ## return index of left/right
    return interval.index(left), interval.index(right)

result = model.predict(components)
print('Predicted Ids (Ox axis coordinates): ')
print('range: 0.2 - 2.6 A')
csv_data = [[] for i in range(len(result))]
for i, pred in enumerate(result):
    left, right = border_curves(round(pred), y)
    print('PCA_comp %d <=> I = %d A between (%d A, %d A)' % (i, round(pred), y[left], y[right]))
    csv_data[i].append(round(pred))
    csv_data[i].append(y[left])
    csv_data[i].append(y[right])

df_pred = df(data= csv_data)
df_pred.to_csv('SRM_RidgePcaPoints.csv')


# Det Nr. Minim puncte  generatoare - Cubic spline interpolation

In [12]:
#cubic spline interpolation

from scipy.interpolate import interp1d
from scipy import interpolate
import itertools
import sys
from pandas import DataFrame as df

def data_from_file2(file_name):
    #case = 1-> Id from file. Iq in 0->220 > 'i_direct_flux'
    #case = 2 -> Iq from file. Id in 0->220 > 'j_q_flux'
    data = np.loadtxt(file_name)
    alfa = list(range(0,31))
    flux = data[0:, :]
    I_currents = np.array([0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6])
    xLabel = " I [A]"
    yLabel = "Flux"
    return alfa, flux, I_currents, xLabel, yLabel
    
alfa, Flux, I_currents, xLabel, yLabel = data_from_file2('flux_s.txt')

def sum_squared_error(gr, pred):
    gr = np.array(gr)
    pred = np.array(pred)
    return (10 ** 6) * (1 / gr.shape[0]) * sum((gr - pred) ** 2)

curve_indices = list(range(0,len(alfa)))
curves_for_plot = [1]# list(range(1, 13,2))
pts_for_plot = [5,4,3]

# try interpolating with only 10, 9, 8 ... 4, 3 points
sub_points_num  = list(range(len(I_currents)-1, 2, -1))

headers = ['alfa'] + ['pts num: %d' % pts for pts in sub_points_num] + ['    Best 5 Points   ']
rows = []
x_best_5_pts = []
y_best_5_pts = []

#define total error per no of points
total_error_per_points_num = np.zeros([31,13])

#dataframe for error table latex
# error_latex = np.zeros(shape = (13,13))

for curve in curve_indices: #pentru fiecare curba alfa
    y = Flux[:, curve] #iau fluxurile corespunzatoare valorii lui alfa
    x = I_currents #iau id-urile
        
    tck = interpolate.splrep(x, y, s=0) #face spline pe toate cele 13 puncte din I_currents
    x_new = np.linspace(min(x), max(x), 200) #generez mai multe valori pentru interpolare
        
    # baseline = y values obtained when interpolating with all ground truth points
    baseline = interpolate.splev(x_new, tck, der=0)
    
    row = ['alfa %d gr' % (alfa[curve])]
    best_5_points =[]
   
    for points_num in sub_points_num:
        # store best configuration
        best_pts     = (x, y)
        best_interp  = (x_new, baseline)
        best_error   = sys.float_info.max
        best_err_gr  = sys.float_info.max
    
        sub_points_sets = itertools.combinations(range(len(I_currents)), points_num)
        for pts in sub_points_sets:
            pts = list(pts)
            x_     = x[pts]
            y_     = y[pts]
                     
            if points_num == 3:
                tck_   = interpolate.splrep(x_, y_, s=0, k=2)
                y_new_ = interpolate.splev(x_new, tck_, der=0) 
            else:
                tck_   = interpolate.splrep(x_, y_, s=0)
                y_new_ = interpolate.splev(x_new, tck_) 
        
            # error -- difference between baseline interp curve and current one
            error = sum_squared_error(baseline, y_new_)
            
            if error < best_error:
                best_error  = error
                best_pts    = (x_, y_) # best points from combination
                best_interp = (x_new, y_new_) # best points from interpolation
                
                # interpolate in target points + evaluate error
                gr_interp = interpolate.splev(x, tck_, der=0)
                best_err_gr = sum_squared_error(y, gr_interp)
        
        if points_num == 5:
            best_5_points = best_pts[0]
            x_best_5_pts.append(best_pts[0])#to be used for linear interp
            y_best_5_pts.append(best_pts[1])#to be used for linear interp
            
        row.append(best_err_gr)
      
    #         error_latex[curve][points_num] = best_err_gr
        
        total_error_per_points_num[points_num] += best_err_gr
     
    row.append(best_5_points)
    rows.append(row)

#tabel csv for best_5_points
df_best_5_points = df(data = x_best_5_pts,
                      index = alfa,
                      columns = list(range(1,6))                                       
                     )
# print(df_best_5_points)
df_best_5_points.to_csv('SRM_best_5_points.csv')

# error_latex = np.around(error_latex, decimals=2)  

# df_error_latex = df(data = error_latex[:,3:8],
#                    index = alfa,
#                    columns = list(range(3,8)))

# df_error_latex.to_csv('SRM_interp_error_points.csv')
    
    
#plotting the errors
    
# x_axis = list(range(3,12))
# plt.clf()
# plt.rcParams["figure.figsize"] = [12, 8]
# plt.rcParams['xtick.labelsize']=14
# plt.rcParams['ytick.labelsize']=14

# # plt.title('Spline interpolation error for different number of points',fontsize=18)
# plt.scatter(x_axis,total_error_per_points_num[3:12]/12)
# plt.plot(x_axis,total_error_per_points_num[3:12]/12)
# plt.xlabel('No of points',fontsize=18)
# plt.ylabel('Spline interpolation error',fontsize=18)
# # plt.ylim([0,2])
# plt.grid()
# plt.savefig('interp_error_Points_graphic.png', dpi = 200)
# plt.show()

print('Interpolation error ( sum squared error * 10 ^ 6)')
table = plotTable(headers=headers, rows=rows, column_width=True, precision=5)
print(table)


Interpolation error ( sum squared error * 10 ^ 6)
+------------+-------------+-------------+-------------+------------+------------+------------+------------+------------+------------+------------+----------------------+
|    alfa    | pts num: 12 | pts num: 11 | pts num: 10 | pts num: 9 | pts num: 8 | pts num: 7 | pts num: 6 | pts num: 5 | pts num: 4 | pts num: 3 |      Best 5 Points   |
+============+=============+=============+=============+============+============+============+============+============+============+============+======================+
| alfa 0 gr  | 0.00000     | 0.00001     | 0.00020     | 0.00493    | 0.01877    | 0.05517    | 0.06046    | 0.96847    | 12.00729   | 189.81039  | [0.8 1.  1.4 2.2     |
|            |             |             |             |            |            |            |            |            |            |            | 2.4]                 |
+------------+-------------+-------------+-------------+------------+------------+------------+

# Det Nr. Minim puncte  generatoare - Cubic spline interpolation

In [53]:
#
import sys
from scipy.interpolate import interp1d
from scipy import interpolate
import numpy as np
from pandas import DataFrame as df

best_points = [0.4,1.0,1.8,2.2,2.6]
best_indices = [1,4,8,10,12] #indices of best-points
all_curves = list(range(31))
curves_for_plot = [11,6]

#indices of alfa curves
config_curves = [[0,30],list(range(0,31,15)),list(range(0,31,10)),list(range(0,31,7)),list(range(0,31,6)),
                 list(range(0,31,5)),list(range(0,31,4)),list(range(0,31,3)),list(range(0,31,2)),#
                 [0, 2, 4, 5, 6, 8, 10, 12, 14,15, 16, 18, 20, 22, 24,25, 26, 28, 30],[],[1,2,3],[1,2]]

error_per_config = np.zeros(shape = len(config_curves))

# headers = ['%s' %Is] + ['no of curves: %d' % len(nr)for nr in config_curves]+['best no of curves']+['######### best config #########']
headers = ['######### Configuration ##########'] + ['No of curves'] + ['error_per_config']#['   alfa %d   ' % alfa[index] for index in all_curves] + ['error_per_config']
rows    = []
#define errors per curve in each config
error_per_curve = np.zeros(shape=(len(all_curves),len(config_curves)))

#define total error per config
error_per_config = np.zeros(shape = len(config_curves))

#define error for latex tables

# flux pt best_points
def flux_for_curves_config(curves_config_indices):
    flux = [[] for i in range(31)]
    for curve in curves_config_indices:
        for idx in best_indices:
            flux[curve].append(Flux[idx][curve])
    return flux    

#incadrez curba intre doua curbe alfa din config

def border_curves(curve, interval):
    pos = 0
    for id in interval:
        if alfa[curve] > alfa[id]:
            pos+=1
        else:
            if alfa[curve] == alfa[id]:
                if pos+1 == len(interval):
                    return pos, pos-1
                else:
                    return pos, pos+1
 
    return pos-1, pos

def sum_squared_error(gr, pred):
    gr = np.array(gr)
    pred = np.array(pred)
    return (10**3 ) * (1 / gr.shape[0]) * sum((gr - pred) ** 2)

for idx in range(len(config_curves)): #iau o configuratie
    indices = config_curves[idx]
    row =[config_curves[idx]]
    row.append(len(config_curves[idx]))  
  
    for curve in all_curves: #pt fiecare curba
        start, end = border_curves(curve, indices)  #gasesc curbele care imi incadreaza curba de aproximat
        flux = flux_for_curves_config([start, end]) # retin fluxurile pentru curbele din config

        #interpolare liniara                       
        #alpha= from interp formula
        #alfa = my curves name
        
        alpha = 1-((alfa[curve]-alfa[start])/(alfa[end]-alfa[start]))
             
        flux_approx = [] # spline interpolate the current curve between the curves in the config
        for i in range(5):
            flux_approx.append(alpha * flux[start][i] + (1-alpha)*flux[end][i])

        ##Spline##
        
        #plotez cu toate cele 13 pcte
        y = Flux[:, curve] #iau fluxurile corespunzatoare
        x = I_currents #iau pctele de pe Ox - I-currents

        tck = interpolate.splrep(x, y, s=0) #fac spline pe toate cele 13 puncte
        x_new = np.linspace(min(x), max(x), 200) #creez mai multe puncte

        # baseline = y values obtained when interpolating with all ground truth points
        baseline = interpolate.splev(x_new, tck, der=0) 
              
        # interpolate in best x points
        x_     = best_points
        y_     = flux_approx
        tck_   = interpolate.splrep(x_, y_, s=0)
        y_new_ = interpolate.splev(x_new, tck_, der=0)            
        
      # error -- difference between baseline interp curve and current one
        error = sum_squared_error(baseline, y_new_)   
           
        error_per_curve[curve][idx] = error
        error_per_config[idx]+=error
        
#         row.append(error)
         
    row.append(error_per_config[idx]/12)
    
    rows.append(row) 
    

print('Interpolation error ( sum squared error * 10 ^ 3)')
table = plotTable(headers=headers, rows=rows, column_width=True, precision=3)
print(table)


Interpolation error ( sum squared error * 10 ^ 3)
+------------------------------------+--------------+------------------+
| ######### Configuration ########## | No of curves | error_per_config |
+====================================+==============+==================+
| [0, 30]                            | 2            | 34.233           |
+------------------------------------+--------------+------------------+
| [0, 15, 30]                        | 3            | 33.904           |
+------------------------------------+--------------+------------------+
| [0, 10, 20, 30]                    | 4            | 22.901           |
+------------------------------------+--------------+------------------+
| [0, 7, 14, 21, 28]                 | 5            | 20.432           |
+------------------------------------+--------------+------------------+
| [0, 6, 12, 18, 24, 30]             | 6            | 19.027           |
+------------------------------------+--------------+------------------+
|

In [25]:
def border_curves(curve, interval):
    pos_in_interval = 0
    for id in interval:
#         print(alfa[id])
        if alfa[curve] > alfa[id]:
            pos_in_interval+=1
        else:
            if alfa[curve] == alfa[id]:
                if pos_in_interval+1 == len(interval):
                    return pos_in_interval, pos_in_interval-1
                else:
                    return pos_in_interval, pos_in_interval+1
         
    return interval[pos_in_interval-1], interval[pos_in_interval ]
start, end = border_curves(3,[4,6,9])
print('start = %d, end = %d'%(start, end))
print('alfa start = %d, end = %d'%(alfa[start], alfa[end]))
start, end = border_curves(5,[4,6,9])
print('start = %d, end = %d'%(start, end))
print('alfa start = %d, end = %d'%(alfa[start], alfa[end]))
start, end = border_curves(8,[4,6,9])
print('start = %d, end = %d'%(start, end))
print('alfa start = %d, end = %d'%(alfa[start], alfa[end]))

start = 9, end = 4
alfa start = 9, end = 4
start = 4, end = 6
alfa start = 4, end = 6
start = 6, end = 9
alfa start = 6, end = 9


# PCA for reducing the number of curves - alfa


In [ ]:
#
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy import interpolate
from sklearn.decomposition import PCA
from sklearn import preprocessing
from pandas import DataFrame as df


#preprocess the data
flux = Flux.T
scaler = preprocessing.StandardScaler()
df_flux_scaled = scaler.fit_transform(flux)
weights  = []

no_of_comps = [13]

#table - average weights of PCs  
avg_weights_headers = ['Components'] + ['Alfa %d' % iq for iq in alfa]
avg_weights_rows = []

for comp in no_of_comps: #for each component
    pca_reduce_curves = PCA(n_components=comp)
    flux_reduced = pca_reduce_curves.fit_transform(df_flux_scaled)
    projected  = pca_reduce_curves.inverse_transform(flux_reduced)
    projected_unscaled = scaler.inverse_transform(projected)

    
    # plot main components
    plt.clf()
    plt.rcParams["figure.figsize"] = [12,8]
    plt.rcParams['xtick.labelsize']=12
    plt.rcParams['ytick.labelsize']=12
    
    components = pca_reduce_curves.components_
    components = scaler.inverse_transform(components)
        
    for idx in range(components.shape[0]):
        plt.plot(I_currents, components[idx, :], label='Principal Component %d'%idx)
        plt.xlabel(xLabel,fontsize=14)
        plt.ylabel(yLabel,fontsize=14)
        plt.legend(fontsize=12)  
        plt.ylim([0,0.35])
        plt.xlim([0, 2.65])
        plt.xticks(I_currents)
         
#     avg_weights = np.mean(abs(pca_reduce_curves.components_), axis=0)
#     weights.append(avg_weights.tolist())
#     avg_Iqs_weights_rows.append([comp] + avg_weights.tolist())
   
    
    plt.grid()
    plt.savefig('SRM_TOP{}PrincipalComponents.png'.format(comp), dpi = 200)  
    plt.show() 
        
# print PC meaning   
# print ("Meaning of the %d components(Weights for each IQ curve)"  %comp)
# table_PC = plotTable(headers=headers_meaning, rows=rows_meaning, column_width=True, precision=5) 
# print(table_PC)

# Interpret PCs for reducing the number of curves - Ridge Regression

In [ ]:
#
from sklearn.linear_model import Ridge
from pandas import DataFrame as df

X = Flux.T
y = alfa

model = Ridge(fit_intercept=True, normalize=True)
model.fit(X, y)

# best case 1, worst case -1, random prediction: 0
print('Regression train score: %.2f\n' % model.score(X, y))

components = pca_reduce_curves.components_
components = scaler.inverse_transform(components)

result = model.predict(components)
print('Predicted alfa: ')
for i, pred in enumerate(result):
    print('PCA_comp %d <=> alfa= %d grades' % (i, round(pred)))


#find upper and lower Iq curve
def border_curves(curve, interval):
    left = max([i for i in interval if i < curve])
    right = min([i for i in interval if i > curve])

    ## return index of left/right
    return interval.index(left), interval.index(right)


plt.clf()
plt.rcParams["figure.figsize"] = [12, 8]
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12

top = 0
xAxis = list(I_currents))

#data for csv
ridge_for_csv = [[] for i in range(len(result))]


for comp in components:    
    pred_iq = model.predict(comp.reshape(1, -1))
    left, right = border_curves(pred_iq, alfa)
    plt.title('Principal component : %d' % (top+1),fontsize=18)
    plt.plot(I_currents, flux[:, left], label='Iq = %d A' % alfa[left])
    plt.plot(
        I_currents,
        components[top],
        color='black',
        label='Iq predicted for comp no %d = %d A' % (top + 1, pred_iq))
    plt.plot(I_currents, flux[:, right], label='Iq = %d A' % alfa[right])
    plt.legend(fontsize=14)
    plt.xlabel(xLabel, fontsize = 14)
    plt.ylabel(yLabel, fontsize = 14)
    plt.xticks(list(range(0, 240, 20)))
    plt.xlim(0, 220)
    plt.grid()
    plt.savefig('SRM_PCA_comp{}_Ridge.png'.format(top+1), dpi = 220)
    plt.show()
    
    # prepare data for csv
    
#     ridge_for_csv[top].append(round(pred_iq[0]))
#     ridge_for_csv[top].append(iqs[left])
#     ridge_for_csv[top].append(iqs[right])
    
#     top += 1

# save to csv
# df_ridge_to_csv_curves = df(data = ridge_for_csv)
# df_ridge_to_csv_curves.to_csv('SRM_RidgePcaCurves.csv')
# print(df_ridge_to_csv_curves)
# print(ridge_for_csv)
